In [1]:
import requests
import pandas as pd
import json
from mstrio.connection import Connection
from mstr_robotics import chat_GPT_for_qs,mstr_classes,report
from mstrio.api import objects
from mstrio.api import languages
i_translate=chat_GPT_for_qs.translate()
i_cube=report.cube()
i_md_searches= mstr_classes.md_searches()
i_obj=objects

In [ ]:
import os
base_url="http://85.214.60.83:8080/MicroStrategyLibrary"
username="Administrator"
password="Vicky_01"
project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
cube_folder_id="1F7ADC49433A122C59E398858E797087"
#conn=get_conn(base_url,project_id=project_id,username=username,password=password)
conn = Connection(base_url=base_url,username=username,password=password,project_id=project_id)
conn.headers['Content-type'] = "application/json"
sKey=os.environ.get("OPENAI_API_KEY")

In [3]:
objj_l=i_md_searches.search_for_type_l(conn=conn,obj_l=[4],info_level = "base_path")
num_m=3
obj_l=[]
for m in objj_l[:num_m]:
    obj_l.append({"id":m["id"],"type":m["type"]})

obj_l    

[{'id': 'CAAC011C4617A0267D5C0C88039B6916', 'type': '4'},
 {'id': 'F9472D7F467DD618C90DCABA0ABC199B', 'type': '4'},
 {'id': '06EA394D476503D21B193A9CF791F4F7', 'type': '4'}]

In [4]:
all_obj_trans_l=i_translate.extract_translations(conn=conn,obj_l=obj_l,project_id=project_id)

In [5]:
#correct spelling
all_trans_output_l=[]

for word in all_obj_trans_l[:num_m]:
    org_lang=word["default_lang_name"]
    trans_data=word["trans_lang_d"]
    trans_data["gpt_comment"]=""
    trans_data["correct_fg"]=""


    messages=[
       {"role": "system", "content": "Please provide the content as a JSON formated file."} ,
       {"role": "system", "content": f":Please correct the spelling for each word {str(trans_data)} " }
             ]
    
    word["trans_data"]=i_translate.check_trans_chatGPT(sKey=sKey,messages=messages,temperature=0.9)
    all_trans_output_l.append(word.copy())

#all_trans_output_l

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [20]:
output_d=i_translate.gpt_out_trans_j_to_d(all_trans_output_l)
spell_all_df=pd.DataFrame(output_d["clean_read_out"])
spell_diff_df=pd.DataFrame(output_d["gpt_change"])

invalid syntax (<unknown>, line 1)
unterminated string literal (detected at line 1) (<unknown>, line 1)
unterminated string literal (detected at line 11) (<unknown>, line 11)
unterminated string literal (detected at line 13) (<unknown>, line 13)
'{' was never closed (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
unterminated string literal (detected at line 11) (<unknown>, line 11)
invalid syntax (<unknown>, line 1)
'{' was never closed (<unknown>, line 1)
unterminated string literal (detected at line 11) (<unknown>, line 11)
invalid syntax (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
unterminated string literal (detected at line 7) (<unknown>, line 7)
unterminated string literal (detected at line 7) (<unknown>, line 7)
unterminated string literal (detected at line 7) (<unknown>, line 7)
unterminated string literal (detected at line 1) (<unknown>, line 1)
unterminated string literal (detected at line 8) (<unknown>, line 8)
unterminat

In [21]:
cbe_spell_all_id=None
cbe_spell_diff_id=None
cube_folder_id="1699E8B64FC423481FC1BABB849F7103" #Schema Monitor

spell_all_d_l=[{"df":spell_all_df,"tbl_name":"spell_all_df", "update_policy":"Replace"}]
spell_diff_d_l=[{"df":spell_diff_df,"tbl_name":"spell_diff_df", "update_policy":"Replace"}]
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=spell_all_d_l,mtdi_id=cbe_spell_all_id , cube_name="spell_all",folder_id=cube_folder_id,force=True)
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict =spell_diff_d_l,mtdi_id=cbe_spell_diff_id , cube_name="spell_diff",folder_id=cube_folder_id,force=True)

Created super cube 'spell_all' with ID: 'E525ADAF4A63F123312FCAB7A13EAAD2'.


  0%|          | 0/1 [00:00<?, ?it/s]

Super cube 'spell_all' published successfully.
Created super cube 'spell_diff' with ID: '63B87A074ECBEC9EBDD44487E83D4BE3'.


  0%|          | 0/1 [00:00<?, ?it/s]

Super cube 'spell_diff' published successfully.


'63B87A074ECBEC9EBDD44487E83D4BE3'

In [ ]:
#tranlations
all_trans_output_l=[]

for word in all_obj_trans_l[:num_m]:
    org_lang=word["default_lang_name"]
    trans_data=word["trans_lang_d"]
    trans_data["gpt_comment"]=""
    trans_data["correct_fg"]=""
    trans_data["special_char_fg"]=""

    check_cont= f": The base language is English (United States)"

    messages=[
      {"role": "system", "content":  f"Please correct the following translations {str(trans_data)} for each language." },
      {"role": "system", "content": "Your output must be a correct formated JSON  file. this"} ,
      {"role": "system", "content": "Please put comment into gpt_comment, if it is really neccesarry."}
             ]
    
    word["trans_data"]=i_translate.check_trans_chatGPT(sKey=sKey,messages=messages,temperature=0.4)
    all_trans_output_l.append(word.copy())



In [ ]:
output_d=i_translate.gpt_out_trans_j_to_d(all_trans_output_l)
trans_all_df=pd.DataFrame(output_d["clean_read_out"])
trans_diff_df=pd.DataFrame(output_d["gpt_change"])

In [27]:

for index,trans in update_df.iterrows():
    trans_d={"op": "REPLACE",
             "path": f"/localesAndTranslations/{trans['text_field_key']}/translationValues/{trans['code']}/translation",
             "value": "SuperWucher"}
    operation_l.append(trans_d.copy())
    body={
  "operationList": [
    {
      "op": "REPLACE",
      "path": "/localesAndTranslations/MTowOi0zODE5ODBCMjExRDQwQkM4QzAwMEM4OTA2Qjk4NDk0Ri00/translationValues/1031/translation",
      "value": "SuperWucher"
    }
  ]
}

[{'op': 'REPLACE',
  'path': '/localesAndTranslations/MTowOi0zODE5ODBCMjExRDQwQkM4QzAwMEM4OTA2Qjk4NDk0Ri00/translationValues/1031/translation',
  'value': 'SuperWucher'},
 {'op': 'REPLACE',
  'path': '/localesAndTranslations/MTowOi0yOTU5NjQ3MDQxQTU5NTdDNjUwQUNEOTk3MDdGNDZCQi00/translationValues/1033/translation',
  'value': 'SuperWucher'},
 {'op': 'REPLACE',
  'path': '/localesAndTranslations/MTowOi0zMDdBMjU5OTQ0QjA1RkREMUZBQTQxODk2NDAxNTAwOS00/translationValues/1033/translation',
  'value': 'SuperWucher'},
 {'op': 'REPLACE',
  'path': '/localesAndTranslations/MTowOi0zM0M1OEFGMjQxOTVFMEEzMUY5NTY4QTkxMjYyQTk2RC00/translationValues/1033/translation',
  'value': 'SuperWucher'},
 {'op': 'REPLACE',
  'path': '/localesAndTranslations/MTowOi00MjQ4MDU1QzQxM0U5Q0YxRkI2QjNCQjBBNjFBMTU1MS00/translationValues/1033/translation',
  'value': 'SuperWucher'},
 {'op': 'REPLACE',
  'path': '/localesAndTranslations/MTowOi00ODRERDg1NjRENTZDMzczRDU4MDM0OUE4RDc5MTQ2RC00/translationValues/1033/translation',
 

In [58]:
#update MD
excel_file="D:\\shared_drive\\demo_data\\MSTR_Spelling.xlsx"
update_df=pd.read_excel(excel_file)

for index,trans in update_df.iterrows():
    operation_l=[]
    trans_d={"op": "REPLACE",
             "path": f"/localesAndTranslations/{trans['text_field_key']}/translationValues/{trans['code']}/translation",
             "value": trans["translation"]}

    body_d=json.dumps({"operationList": [trans_d]})
    
    #print(body_d)
    url=f"{conn.base_url}/api/objects/metric/{trans['obj_id']}/translations"
    new_trans=conn.patch(url,data=body_d)
    print(new_trans.json())


{'defaultLanguage': 'en', 'localeName': 'English (United States)', 'localesAndTranslations': {'MTowOi0zODE5ODBCMjExRDQwQkM4QzAwMEM4OTA2Qjk4NDk0Ri00': {'translationTargetName': 'Object Name ', 'translationValues': {'1040': {'translation': 'Sconto'}, '1041': {'translation': 'ディスカウント'}, '1042': {'translation': '할인'}, '1028': {'translation': '折扣'}, '2052': {'translation': '折扣'}, '1046': {'translation': 'Desconto'}, '1031': {'translation': 'Rabatt'}, '1033': {'translation': 'Discount'}, '3082': {'translation': 'Descuentu'}, '1036': {'translation': 'Remise'}, '1053': {'translation': 'Diskontering'}}}}}
{'defaultLanguage': 'en', 'localeName': 'English (United States)', 'localesAndTranslations': {'MTowOi0yOTU5NjQ3MDQxQTU5NTdDNjUwQUNEOTk3MDdGNDZCQi00': {'translationTargetName': 'Object Name ', 'translationValues': {'1040': {'translation': 'Costi fissi mensili'}, '1041': {'translation': '月次固定コスト'}, '1042': {'translation': '월별 고정 비용'}, '1028': {'translation': '每月固定成本'}, '2052': {'translation': '每

In [ ]:
import PyPDF2
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO

# Function to fake text fields in a PDF
def fake_text_fields(input_pdf_path, output_pdf_path):
    # Read the input PDF
    input_pdf = PyPDF2.PdfFileReader(open(input_pdf_path, 'rb'))
    output_pdf = PyPDF2.PdfFileWriter()
    
    # Create a buffer to store the faked text
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    
    # Iterate through all the pages
    for page_num in range(input_pdf.getNumPages()):
        page = input_pdf.getPage(page_num)
        content = page.extractText()
        
        # Generate faked text for each page
        faked_text = "FAKED TEXT"
        
        # Draw the faked text on the page
        can.drawString(100, 750, faked_text)
        can.showPage()
        
        # Add the original content (optional, if you want to keep it)
        can.drawString(100, 700, content)
    
    can.save()
    
    # Move to the beginning of the StringIO buffer
    packet.seek(0)
    new_pdf = PyPDF2.PdfFileReader(packet)
    
    # Merge the faked text with the original PDF
    for page_num in range(input_pdf.getNumPages()):
        page = input_pdf.getPage(page_num)
        faked_page = new_pdf.getPage(page_num)
        page.mergePage(faked_page)
        output_pdf.addPage(page)
    
    # Write the output PDF
    with open(output_pdf_path, 'wb') as output_file:
        output_pdf.write(output_file)

# Usage
input_pdf_path = 'input_dashboard.pdf'
output_pdf_path = 'faked_dashboard.pdf'
fake_text_fields(input_pdf_path, output_pdf_path)
